**Assignment 2 - Question 3**
- Provide the answers in the provided cell.

In [1]:
# Install some dependencies. We will use transformers and datasets package from Huggingface for our experiments. Uncomment the line below to install the dependencies.
!pip install transformers datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.4 MB/s eta 0:00:00a 0:00:01
  Using cached tokenizers-0.13.3-cp310-cp310-macosx_12_0_arm64.whl (3.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.4/401.4 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
  Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.9/343.9 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 22.2 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Load the GSM8K dataset. There are two versions: main and Socratic. We are currently interested in main but later on we can explore socratic version. Just pass the name in the command below to load them.
from datasets import load_dataset
dataset = load_dataset("gsm8k", "main")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [3]:
# We test on 100 samples only for all the experiments
test_samples = dataset["test"][:100]

In [4]:
# Printing a sample from the dataset
# The dataset is a Math Word Problem in the following format. "Ques" represents the main problem and "Solution" contains the step by step solution. The final answer appears after "####"
sample_ques = test_samples["question"][0]
print (f"Ques: {sample_ques}\nSolution: {test_samples['answer'][0]}")

Ques: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Solution: Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


**Ques 1**: We have training dataset available for GSM8K ("train" parameter in dataset) and we start by training an LLM. We use T5 models for all our experiments (T5 and Flan versions). For training, we use T5-small and train it on GSM8K dataset and report the accuracy on the test set as created above.
- For training and testing, use the train and test scripts provided in the GitHub (https://github.com/openai/grade-school-math/tree/master/grade_school_math).
- Make sure to replace the GPT2 tokenizer and model with T5-small from Huggingface (also provided below).
- For testing run sample.py file for all the 100 samples from the test set (from above). No need to modify the calculator code and keep using one.
- Finally, report the accuracy of the model on the 100 test samples created above.

NOTE: No need to submit the train and sample code. Only the results are enough. The results are expected in .ccsv format with questions in one column and model output in the second column. Accuracy is calculated based on the final answer only (the one after ####).

**Answer 1**:

# Example for one device on pre-trained model

In [5]:
#Loading T5 model and analyzing the output out of the box
from transformers import AutoTokenizer, T5ForConditionalGeneration

#Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("t5-small", model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# lets create an input for the model. Here we are testing with one example question out of the box.
input_sample = sample_ques

# Pass the sample to the model and print the decoded output
input_ids = tokenizer(input_sample, return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

and bakes muffins for her friends every day with four. Janet’s ducks lay 16 eggs per day. Janet’s ducks lay 16 eggs per day. Janet sells the remainder at the farmers' market daily for $2 per fresh duck egg.


# Loading necessary functions for sampling from model

In [6]:
from contextlib import contextmanager
import signal
import torch as th

# taken from
# https://stackoverflow.com/questions/492519/timeout-on-a-function-call
@contextmanager
def timeout(duration, formula):
    def timeout_handler(signum, frame):
        raise Exception(f"'{formula}': timed out after {duration} seconds")

    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(duration)
    yield
    signal.alarm(0)


def eval_with_timeout(formula, max_time=3):
    try:
        with timeout(max_time, formula):
            return eval(formula)
    except Exception as e:
        signal.alarm(0)
        print(f"Warning: Failed to eval {formula}, exception: {e}")
        return None


def use_calculator(sample):
    if "<<" not in sample:
        return None

    parts = sample.split("<<")
    remaining = parts[-1]
    if ">>" in remaining:
        return None
    if "=" not in remaining:
        return None
    lhs = remaining.split("=")[0]
    lhs = lhs.replace(",", "")
    if any([x not in "0123456789*+-/.()" for x in lhs]):
        return None
    return eval_with_timeout(lhs)


def sample(model, qn, tokenizer, device, sample_len):
    # Inefficient version of calculator sampling -- no batches, doesn't
    # cache activations from previous tokens
    EQUALS_TOKENS = set([28, 796, 47505])

    for _ in range(sample_len):
        with th.no_grad():
            toks = tokenizer([qn], padding=False, return_tensors="pt").to(device)
            orig_len = toks["input_ids"].shape[1]

            out = model.generate(
                **toks, max_length=orig_len + 1, pad_token_id=model.config.eos_token_id
            )
            text = tokenizer.batch_decode(out)[0]

            if out[0, -1].item() in EQUALS_TOKENS:
                answer = use_calculator(text)
                if answer is not None:
                    print("Triggered calculator, answer", answer)
                    text = text + str(answer) + ">>"

            qn = text
    return qn


In [11]:
import os
os.getcwd()

'/Users/qguilhot/Library/Mobile Documents/com~apple~CloudDocs/ETHZ/Large language models/Assignment_2/LLM_2/grade_school_math'

# Loading the True model pre-trained

In [12]:
device = th.device("mps")
tokenizer = AutoTokenizer.from_pretrained("t5-small", model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained("../model_ckpts") #change it to "model_ckpts" later
model.to(device)
print("Model Loaded")

Model Loaded


# Getting the list of the questions and results

In [13]:
sample_len = 100
qn = test_samples["question"][:sample_len]
rn = test_samples["answer"][:sample_len]
qn = [q.strip() for q in qn]


# Sampling from the model

In [14]:
sn = [sample(model, q, tokenizer, device, sample_len) for q in qn]

TypeError: Operation 'abs_out_mps()' does not support input type 'int64' in MPS backend.

# Getting Back the True Answers

In [ ]:
import pandas as pd
pred = pd.DataFrame({"questions":qn, "answers":rn,"predictions":sn})

**Ques 2:** Finetuning works well for cases when training data is available. But for cases when there is no training data available, we can first test how the pre-trained models work out of the box in a zero-shot setting.
- Load versions of T5 models [small, base, and large] from [HuggingFace repo](https://huggingface.co/docs/transformers/model_doc/t5) as demonstrated in the example above and run the inference for entire test set of 100 examples. Calculate the accuracy on the test samples and plot a chart demonstrating how the accuracy scales with model sizes.
NOTE: Template to plot the graph in bar chart format is provided at the end of the document.

**Answer 2**:

In [ ]:
#Loading T5 model and analyzing the output out of the box
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Add the model and the tokenizer here

# Modify the input sample

inputs = tokenizer(input_sample, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

**Ques 3**: Without specifying what the model should do for a given problem, the model might feel that the task is a typical sentence completion task which might not be desirable for a question-answering task or machine translation task. *Instruction tuning* is an idea to train these LLMs with instructions for each task which if provided during test time, can assist the model to decide how to approach the task.
- Now repeat the same experiments with Flan T5 models [small, base, and large] (an instruction tuned version of T5) and plot a chart comparing it with the T5 models in the same last plot:
   - without changing the input style and
   - with adding instruction for each input example (Refer to [FlanT5 paper](https://arxiv.org/pdf/2210.11416.pdf) and figure out the instruction that can prove useful for answering GSM8K dataset).
   
   NOTE that all you need is to load the tokenizer and the model in the above code, and provide the input sample in the desired format. Also, the instructions are task specific meaning a common instruction in the input sample will work for all the examples. No need to write different instructions for each example.

**Answer 3**:

**Ques 4**: Often zero shot inference capabilities can be improved by providing some incontext examples to the model (not true for all cases though). For FlanT5 large model, provide 2-3 incontext examples (should be other than the 100 testing samples) and analyze the performance of the models as following:

   * Only provide the final answer (In the example sample above, it will be "The answer is 18").
   * Replace the final answer only to a step by step explaination also refer to chain of thought reasoning (refer to Figure 11 in the [Flan paper](https://arxiv.org/pdf/2210.11416.pdf) in the Appendix).
   * Plot the approaches along with the other two results for FlanT5 model from part 2.

   NOTE: The in-context examples should be appended to the provided input in the input sample.

**Answer 4**:

**BONUS Ques**: Can you come up with another style of step by step reasoning prompts that can prove useful (HINT: check the Socratic version of the dataset.). Test your hypothesis with the same FlanT5 model as above and see the difference.  

**BONUS**:

In [ ]:
# Template to plot graphs.
import matplotlib.pyplot as plt

# Sample data
categories = ['Category1', 'Category2']
values = [20, 35]

# Create a figure and a set of subplots
fig, ax = plt.subplots()

# Plot data
ax.bar(categories, values, color=['blue', 'red'])

# Set the y axis label
ax.set_ylabel('Values')

# Set the chart's title
ax.set_title('Bar Chart')

plt.show()